## 3. 분류기 훈련: efficientnet-b3

In [1]:
#감정 분류기 학습 _ EfficientNet
import torch
import torch.nn as nn
from torchvision import models

class EffEmotionClassifier(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()
        # EfficientNet-b0 불러오기 (사전학습)
        base_model = models.efficientnet_b3(pretrained=True)

        # Feature extractor 부분만 사용
        self.features = base_model.features  #b3: (batch,  1536, 7, 7)   b4: 1792, 7, 7)

        # Adaptive Pooling 추가 (출력 크기 맞추기)
        self.pooling = nn.AdaptiveAvgPool2d(1)  # (batch, 1536, 1, 1)

        # Classifier 정의
        self.classifier = nn.Sequential(
            nn.Flatten(),                    # (batch, 1536)
            nn.Linear(1536, 128),            # EfficientNet-b3의 feature dim = 1536
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)     # (B, 1536, 7, 7)
        x = self.pooling(x)      # (B, 1536, 1, 1)
        x = self.classifier(x)   # (B, num_classes)
        return x


In [ ]:
####### LR scheduler + early stopper
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import copy


# 감정 분류기 학습 파이프라인

## 1. 데이터 전처리 및 로딩

# 경로 설정
train_dir = "/workspace/yoons/data/cropped_images/train"
val_dir = "/workspace/yoons/data/cropped_images/val"

# 이미지 변환 설정
transform = transforms.Compose([
    transforms.Resize((300, 300)),   #((224, 224)),   b4: ((380, 380))
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  #mean=[0.5]
])

# 데이터셋 로드
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)  #bs 전엔 8
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)    #bs 8

class_names = train_dataset.classes

# 2. 모델, loss, optimizer 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EffEmotionClassifier(num_classes=4).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 🔁 LR scheduler 설정: val loss가 줄지 않으면 LR 줄이기
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

# 🛑 Early Stopping 관련 변수
patience = 5
best_val_loss = float('inf')
early_stop_counter = 0
best_model_wts = copy.deepcopy(model.state_dict())

EPOCHS = 50

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)

    train_loss /= len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_acc = correct / len(val_loader.dataset)

    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    # Scheduler step
    scheduler.step(val_loss)

    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print(f"🛑 Early stopping triggered at epoch {epoch+1}")
            break

# 가장 좋은 모델로 복원
model.load_state_dict(best_model_wts)


# 모델 저장
torch.save(model.state_dict(), 'emotion_classifier11_aug_eff_b3_data_added_2.pth')
torch.save(model, 'model_emotion_classifier11_aug_eff_b3_data_added_2.pt')


/root/miniconda3/envs/yoons_env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/envs/yoons_env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B3_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/root/miniconda3/envs/yoons_env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


[Epoch 1] Train Loss: 0.5347 | Val Loss: 0.4389 | Val Acc: 0.8428
[Epoch 2] Train Loss: 0.3296 | Val Loss: 0.4638 | Val Acc: 0.8371
[Epoch 3] Train Loss: 0.2586 | Val Loss: 0.4349 | Val Acc: 0.8521
[Epoch 4] Train Loss: 0.2066 | Val Loss: 0.4634 | Val Acc: 0.8562
[Epoch 5] Train Loss: 0.1688 | Val Loss: 0.4404 | Val Acc: 0.8692
[Epoch 6] Train Loss: 0.1413 | Val Loss: 0.4080 | Val Acc: 0.8766
[Epoch 7] Train Loss: 0.1206 | Val Loss: 0.5884 | Val Acc: 0.8550
[Epoch 8] Train Loss: 0.1015 | Val Loss: 0.5145 | Val Acc: 0.8688
[Epoch 9] Train Loss: 0.0923 | Val Loss: 0.5233 | Val Acc: 0.8554
[Epoch 10] Train Loss: 0.0368 | Val Loss: 0.6296 | Val Acc: 0.8640
[Epoch 11] Train Loss: 0.0278 | Val Loss: 0.7344 | Val Acc: 0.8709
🛑 Early stopping triggered at epoch 11


In [3]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import torch

def evaluate_model(model, dataloader, class_names, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print("Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))


In [4]:
# 클래스 이름 (순서대로!)
class_names = ["분노", "기쁨", "당황", "슬픔"]

# 예: 테스트 dataloader 사용
evaluate_model(model, val_loader, class_names, device)


Confusion Matrix:
[[484   9  37  40]
 [  9 662  12   9]
 [ 49  17 466  34]
 [ 51   7  29 540]]

Classification Report:
              precision    recall  f1-score   support

          분노       0.82      0.85      0.83       570
          기쁨       0.95      0.96      0.95       692
          당황       0.86      0.82      0.84       566
          슬픔       0.87      0.86      0.86       627

    accuracy                           0.88      2455
   macro avg       0.87      0.87      0.87      2455
weighted avg       0.88      0.88      0.88      2455

